2. Pythonではじめるクローリング・スクレイピング

  - 2-1. Pythonを使うメリット
  - 2-2. Pythonのインストールと実行
  - 2-3. Pythonの基礎知識
  - 2-4. Webページを取得する
  - 2-5. Webページからデータを抜き出す
  - 2-6. データをファイルに保存する
  - 2-7. Pythonによるスクレイピングの流れ
  - 2-8. URLの基礎知識
  - 2-9. まとめ

## 2-4. Webページを取得する

### 2-4-1. RequestsによるWebページの取得

In [1]:
import requests

r = requests.get(("https://gihyo.jp/dp"))

In [2]:
# get()関数の戻り値はResponseオブジェクト
type(r)

requests.models.Response

In [4]:
# status_code属性でHTTPステータスコードを取得できる
r.status_code

200

In [5]:
# header属性でHTTPヘッダーの辞書を取得できる
r.headers["content-type"]

'text/html; charset=UTF-8'

In [6]:
# encoding属性でHTTPヘッダーから得られたエンコーディングを取得できる
r.encoding

'UTF-8'

In [7]:
# text属性でstr型にデコードしたレスポンスボディを取得できる
r.text

'<!DOCTYPE HTML>\n<html lang="ja" class="pc">\n<head>\n  <meta charset="UTF-8">\n  <title>Gihyo Digital Publishing … 技術評論社の電子書籍</title>\n  <meta http-equiv="Content-Style-Type" content="text/css"/>\n  <meta http-equiv="Content-Script-Type" content="application/javascript"/>\n  <meta name="description" content="技術評論社の電子書籍（電子出版）販売サイト"/>\n  <meta name="keywords" content="電子書籍,電子出版,EPUB,PDF,技術評論社"/>\n  <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"/>\n  <meta name="apple-mobile-web-app-capable" content="yes"/>\n  <meta name="format-detection" content="telephone=no"/>\n  <link rel="related" href="http://gihyo.jp/dp/catalogs.opds" type="application/atom+xml;profile=opds-catalog" title="Gihyo Digital Publishing OPDS Catalog"/>\n  <link rel="shortcut icon" href="/assets/templates/gdp/favicon.ico" type="image/vnd.microsoft.icon"/>\n  <link rel="apple-touch-icon-precomposed" href="/dp/assets/gdp-icon.png"/>\n  <!--[if lt IE 9]>\n    <script>var msie=8;</script>\n    <script src="/

#### Requestsの高度な機能

In [8]:
# GitHub REST API v3 で、RequestsのリポジトリをJSON形式で取得する
r = requests.get("https://api.github.com/repos/psf/requests")
r.json()

{'id': 1362490,
 'node_id': 'MDEwOlJlcG9zaXRvcnkxMzYyNDkw',
 'name': 'requests',
 'full_name': 'psf/requests',
 'private': False,
 'owner': {'login': 'psf',
  'id': 50630501,
  'node_id': 'MDEyOk9yZ2FuaXphdGlvbjUwNjMwNTAx',
  'avatar_url': 'https://avatars.githubusercontent.com/u/50630501?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/psf',
  'html_url': 'https://github.com/psf',
  'followers_url': 'https://api.github.com/users/psf/followers',
  'following_url': 'https://api.github.com/users/psf/following{/other_user}',
  'gists_url': 'https://api.github.com/users/psf/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/psf/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/psf/subscriptions',
  'organizations_url': 'https://api.github.com/users/psf/orgs',
  'repos_url': 'https://api.github.com/users/psf/repos',
  'events_url': 'https://api.github.com/users/psf/events{/privacy}',
  'received_events_url': 'https://api.github.com/

In [11]:
# httpbin.orgというHTTPリクエスト／レスポンスを試せるサービスを検証に用いる。
# POSTメソッドで送信。
# キーワード引数dataにdictを指定するとHTMLフォーム形式で送信される。
r = requests.post("http://httpbin.org/post", data={"key1": "value1"})
r

<Response [200]>

In [13]:
# リクエストに追加するHTTPヘッダーとキーワード引数headerにdictで指定する。
r = requests.get(
    "http://httpbin.org/get",
    headers={"user-agent": "my-crawler/1.0 (+foo@example.com)"}
)
r

<Response [200]>

In [ ]:
# Basic認証のユーザー名とパスワードの組をキーワード引数authで指定する。
r = requests.get(
    "https://api.github.com.user",
    auth=("<user-id>", "<user-pass>")
)

In [ ]:
# URLのパラメータはキーワード引数paramsで指定することも可能。
r = requests.get("http://httpbin.org/get", params={"key1": "value1"})

In [ ]:
# HTTPヘッダーを複数のリクエストで使い回す
s = requests.Session()
s.headers.update({"user-agent": "my-crawler/1.0 (+foo@example.com)"})

# Sessionオブジェクトにはget(), post()などのメソッドがあり、
# requests.get(), requests.pot()などと同様に使える。
r = s.get("https://gihyo.jp/")
r = s.get("https://gihyo.jp/dp")

### 2-4-2 文字コードの扱い

In [14]:
# UTF-8でエンコードした場合、E3 81 82 という3バイト
"あ".encode("UTF-8")

b'\xe3\x81\x82'

In [15]:
# CP932でエンコードした場合、82 A0 という2バイト
"あ".encode("cp932")

b'\x82\xa0'

In [16]:
# EUC-JPでエンコードした場合、A4 A2 という2バイト
"あ".encode("euc-jp")

b'\xa4\xa2'

#### Webページのエンコーディングを取得・推定する方法

1. HTTPレスポンスのContent-Typeヘッダーのcharsetで指定されたエンコーディングを取得する。
  - 正しくなかったり、charsetがしてされていないことがある。
2. HTTPレスポンスボディのバイト列の特徴からエンコーディングを推定する。
  - 推定に使用するレスポンスボディが長いほど、精度良く推定できる。（処理時間は長くなる）
3. HTMLのmetaタグで指定されたエンコーディングを取得する。
  - 1. が正しくしてされているサイトではmetaタグが指定されていないことがある。

#### HTTPヘッダーからエンコーディングを取得する

- 日本語ページの典型的なContent-Typeヘッダーの値
  - text/html
  - text/html; charset=UTF-8
  - text/html; charset=EUC-JP

In [3]:
import sys
import requests

# 第1引数からURLを取得する
# url = sys.argv[1]
url = "https://gihyo.jp/dp"
# URLで指定したWebページを取得する
r = requests.get(url)
# エンコーディングを標準エラー出力に出力する
print(f"encoding: {r.encoding}", file=sys.stderr)
# デコードしたレスポンスボディを標準出力する
print(r.text)

<!DOCTYPE HTML>
<html lang="ja" class="pc">
<head>
  <meta charset="UTF-8">
  <title>Gihyo Digital Publishing … 技術評論社の電子書籍</title>
  <meta http-equiv="Content-Style-Type" content="text/css"/>
  <meta http-equiv="Content-Script-Type" content="application/javascript"/>
  <meta name="description" content="技術評論社の電子書籍（電子出版）販売サイト"/>
  <meta name="keywords" content="電子書籍,電子出版,EPUB,PDF,技術評論社"/>
  <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"/>
  <meta name="apple-mobile-web-app-capable" content="yes"/>
  <meta name="format-detection" content="telephone=no"/>
  <link rel="related" href="http://gihyo.jp/dp/catalogs.opds" type="application/atom+xml;profile=opds-catalog" title="Gihyo Digital Publishing OPDS Catalog"/>
  <link rel="shortcut icon" href="/assets/templates/gdp/favicon.ico" type="image/vnd.microsoft.icon"/>
  <link rel="apple-touch-icon-precomposed" href="/dp/assets/gdp-icon.png"/>
  <!--[if lt IE 9]>
    <script>var msie=8;</script>
    <script src="//ajax.googleapis.c

encoding: UTF-8


In [5]:
# HTMLをdp.htmlというファイル名で保存
with open("dp.html", "wb") as f:
    f.write(r.content)

#### レスポンスボディのバイト列からエンコーディングを推定する

In [6]:
import sys
import requests

# 第1引数からURLを取得する。
# url = sys.argv[1]
url = "https://gihyo.jp/dp"
# URLで指定したWebページを取得する。
r = requests.get(url)
# バイト列の特徴から推定したエンコーディングを使用する。
r.encoding = r.apparent_encoding
# エンコーディングを標準エラー出力に出力する。
print(f'encoding: {r.encoding}', file=sys.stderr)
# デコードしたレスポンスボディを標準出力に出力する。
print(r.text)


<!DOCTYPE HTML>
<html lang="ja" class="pc">
<head>
  <meta charset="UTF-8">
  <title>Gihyo Digital Publishing … 技術評論社の電子書籍</title>
  <meta http-equiv="Content-Style-Type" content="text/css"/>
  <meta http-equiv="Content-Script-Type" content="application/javascript"/>
  <meta name="description" content="技術評論社の電子書籍（電子出版）販売サイト"/>
  <meta name="keywords" content="電子書籍,電子出版,EPUB,PDF,技術評論社"/>
  <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"/>
  <meta name="apple-mobile-web-app-capable" content="yes"/>
  <meta name="format-detection" content="telephone=no"/>
  <link rel="related" href="http://gihyo.jp/dp/catalogs.opds" type="application/atom+xml;profile=opds-catalog" title="Gihyo Digital Publishing OPDS Catalog"/>
  <link rel="shortcut icon" href="/assets/templates/gdp/favicon.ico" type="image/vnd.microsoft.icon"/>
  <link rel="apple-touch-icon-precomposed" href="/dp/assets/gdp-icon.png"/>
  <!--[if lt IE 9]>
    <script>var msie=8;</script>
    <script src="//ajax.googleapis.c

encoding: utf-8


#### meta タグからエンコーディングを取得する

In [7]:
import sys
import re
import requests

# 第1引数からURLを取得する。
# url = sys.argv[1]
url = "https://gihyo.jp/dp"
r = requests.get(url)  # URLで指定したWebページを取得する。

# charsetはHTMLの最初のほうに書かれていると期待できるので、
# レスポンスボディの先頭1024バイトをASCII文字列としてデコードする。
# ASCII範囲外の文字はU+FFFD（REPLACEMENT CHARACTER）に置き換え、例外を発生させない。
scanned_text = r.content[:1024].decode('ascii', errors='replace')

# デコードした文字列から正規表現でcharsetの値を抜き出す。
match = re.search(r'charset=["\']?([\w-]+)', scanned_text)
if match:
    r.encoding = match.group(1)  # charsetが見つかった場合は、その値を使用する。
else:
    r.encoding = 'utf-8'  # charsetが明示されていない場合はUTF-8とする。

print(f'encoding: {r.encoding}', file=sys.stderr)  # エンコーディングを標準エラー出力に出力する。
print(r.text)  # デコードしたレスポンスボディを標準出力に出力する。

<!DOCTYPE HTML>
<html lang="ja" class="pc">
<head>
  <meta charset="UTF-8">
  <title>Gihyo Digital Publishing … 技術評論社の電子書籍</title>
  <meta http-equiv="Content-Style-Type" content="text/css"/>
  <meta http-equiv="Content-Script-Type" content="application/javascript"/>
  <meta name="description" content="技術評論社の電子書籍（電子出版）販売サイト"/>
  <meta name="keywords" content="電子書籍,電子出版,EPUB,PDF,技術評論社"/>
  <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"/>
  <meta name="apple-mobile-web-app-capable" content="yes"/>
  <meta name="format-detection" content="telephone=no"/>
  <link rel="related" href="http://gihyo.jp/dp/catalogs.opds" type="application/atom+xml;profile=opds-catalog" title="Gihyo Digital Publishing OPDS Catalog"/>
  <link rel="shortcut icon" href="/assets/templates/gdp/favicon.ico" type="image/vnd.microsoft.icon"/>
  <link rel="apple-touch-icon-precomposed" href="/dp/assets/gdp-icon.png"/>
  <!--[if lt IE 9]>
    <script>var msie=8;</script>
    <script src="//ajax.googleapis.c

encoding: UTF-8


### 2-5. Webページからデータを抜き出す

- 正規表現
  - HTMLを単純な文字列をみなして必要な部分を抜き出す
- HTMLパーサー
  - HTMLのタグを解析（パース）して必要な部分を抜き出す

#### 2-5-1 正規表現によるスクレイピング

##### re モジュールの使い方

- https://docs.python.org/3/library/re.html

In [1]:
import re

In [2]:
# re.search()
# 第2引数の文字列が第1引数の正規表現にマッチするかどうかをテスト
re.search(r"a.*c", "abc123DEF")

<re.Match object; span=(0, 3), match='abc'>

In [3]:
re.search(r"a.*d", "abc123DEF")

In [4]:
# 第3引数にオプションを指定する
# re.IGNORECASE, re.I
re.search(r"a.*d", "abc123DEF", re.IGNORECASE)

<re.Match object; span=(0, 7), match='abc123D'>

In [5]:
# Matchオブジェクトのgroup()メソッドでマッチした値を取得できる
# 引数に 0 を指定すると、正規表現全体にマッチした値が得られる
m = re.search(r"a(.*)c", "abc123DEF")
m.group(0)

'abc'

In [6]:
# 引数に 1 以上の数値を指定すると、正規表現の()で囲った部分（キャプチャ）にマッチした値を取得できる
# 1 なら 1 番目のキャプチャに、2 なら 2 番目のキャプチャにマッチした値が得られる
m.group(1)

'b'

In [8]:
# re.findall()
# 正規表現にマッチするすべての箇所を取得する
# \w は、Unicodeで単語の一部になりえる文字にマッチする
# \s は、空白にマッチする
re.findall(r"\w{2,}", "This is a pen")

['This', 'is', 'pen']

In [9]:
# re.sub()
# 正規表現にマッチする箇所を置換する
# 第3引数の文字列の中で、第1引数の正規表現にマッチする箇所（この例では、2文字以上の単語）すべてを、
# 第2引数の文字列に置換した文字列を取得する
re.sub(r"\w{2,}", "That", "This is a pan")

'That That a That'

##### re モジュールを使ったスクレイピング

In [9]:
# 正規表現によるスクレイピング
# scrape_re.py
import re
from html import unescape
from urllib.parse import urljoin

with open("dp.html") as f:
    html = f.read()

# re.findall()を使って、書籍1冊に相当する部分のHTMLを取得する。
# *?は、*と同様だが、なるべく短い文字列にマッチする（non-greedyである）ことを表すメタ文字。
for partial_html in re.findall(r'<a itemprop="url".*?</ul>\s*</a></li>', html, re.DOTALL):
    # 書籍のURLは、itemprop="url"という属性を持つa要素のhref属性から取得する。
    url = re.search(r'<a itemprop="url" href="(.*?)">', partial_html).group(1)
    # 相対URLを絶対URLに変換する。
    url = urljoin("https://gihyo.jp/", url)

    # 書籍のタイトルはitemprop="name"という属性を持つp要素から取得する。
    # p要素全体を取得する。
    title = re.search(r'<p itemprop="name".*?</p>', partial_html).group(0)
    # brタグをスペースに置き換える
    title = title.replace('<br/>', '')
    title = re.sub(r'<.*?>', '', title)
    # 文字参照が含まれている場合は元に戻す。
    title = unescape(title)

    print(url, title)


https://gihyo.jp/dp/ebook/2023/978-4-297-13486-0 今すぐ使えるかんたん 今すぐ使えるかんたんぜったいデキます！ パワーポイント超入門［Office 2021／Microsoft 365両対応］
https://gihyo.jp/dp/ebook/2023/978-4-297-13504-1 ゼロからはじめる ゼロからはじめるドコモ arrows N F-51C スマートガイド
https://gihyo.jp/dp/ebook/2023/978-4-297-13462-4 知りたい！サイエンス ゼータへの最初の一歩 ベルヌーイ数～「べき乗和」と素数で割った「余り」の驚くべき関係～
https://gihyo.jp/dp/ebook/2023/978-4-297-13442-6 今すぐ使えるかんたん 今すぐ使えるかんたんiPad完全ガイドブック 困った解決&便利技［iPadOS 16対応版］
https://gihyo.jp/dp/ebook/2023/978-4-297-13490-7 売れるランディングページ改善の法則
https://gihyo.jp/dp/ebook/2023/978-4-297-13241-5 ディープラーニングG検定（ジェネラリスト） 法律・倫理テキスト
https://gihyo.jp/dp/ebook/2023/978-4-297-13496-9 エンジニア選書 実践 Svelte入門
https://gihyo.jp/dp/ebook/2023/978-4-297-13510-2 現場が動きだす大学教育のマネジメントとは―茨城大学「教育の質保証」システム構築の物語
https://gihyo.jp/dp/ebook/2023/978-4-297-13508-9 Minecraftオフィシャルブック マインクラフト モブのたくらみ［石の剣のものがたりシリーズ②］
https://gihyo.jp/dp/ebook/2023/978-4-297-13408-2 図解即戦力 図解即戦力脱炭素のビジネス戦略と技術がこれ1冊でしっかりわかる教科書
https://gihyo.jp/dp/ebook/2023/978-4-297-13436-5 ノンプログラマーのための Visual Studio 

In [10]:
# 文字列参照
from html import unescape

unescape('クローリング&amp;スクレイピング')

'クローリング&スクレイピング'

#### 2-5-2 XPathとCSSセレクター

#### 2.5.3 lxmlによるスクレイピング

In [2]:
import lxml.html

# parse()関数でファイルを指定してパースする。
tree = lxml.html.parse('dp.html')
type(tree)

lxml.etree._ElementTree

In [ ]:
# ファイルオブジェクトを指定してパースすることも可能
# tree = lxml.html.parse(open('dp.html'))

In [3]:
# getroot()メソッドでhtml要素に対応するHtmlElementオブジェクトが得られる。
html = tree.getroot()
type(html)

lxml.html.HtmlElement

In [11]:
# fromstring()関数で文字列（str型またはbytes型）をパースできる。
# """~"""（'''~'''）囲まれた部分は複数行文字列リテラルで、
# 改行も含めて１つの文字列と解釈される。
# encodingが指定されたXML宣言を含むstrをパースすると、valueErrorが発生する。
html = lxml.html.fromstring("""
    <html>
    <head><title>八百屋オンライン</title></head>
    <body>
    <h1 id=main><strong>おいしい</strong>今日のくだもの</h1>
    <ui>
        <li>りんご</li>
        <li class="featured">みかん</li>
        <li>ぶどう</li>
    </ui>
    </html>
    </body>
""")
# fromstring()関数では直接HtmlElementオブジェクトが得られる。
type(html)

lxml.html.HtmlElement

In [5]:
# HtmlElementのxpath()メソッドでXPathにマッチする要素のリストが取得できる。
html.xpath('//li')

[<Element li at 0x102d0a6b0>,
 <Element li at 0x10473a430>,
 <Element li at 0x1047394e0>]

In [6]:
# cssselect()メソッドでCSSセレクターにマッチする要素のリストが取得できる。
html.cssselect('li')

[<Element li at 0x102d0a6b0>,
 <Element li at 0x10473a430>,
 <Element li at 0x1047394e0>]

In [7]:
# さまざまなセククターで絞り込み可能
html.cssselect('li.featured')

[<Element li at 0x10473a430>]

In [13]:
# h1要素を取得する。
h1 = html.cssselect('h1')[0]
# tag属性でタグの名前を取得できる。
h1.tag

'h1'

In [14]:
# get()メソッドで属性の値を取得できる。
h1.get('id')

'main'

In [15]:
# attrib属性で全属性を表すdict-likeなオブジェクトを取得できる。
h1.attrib

{'id': 'main'}

In [16]:
# getparent()メソッドで親要素を取得できる。
h1.getparent()

<Element body at 0x10ab859e0>

In [18]:
# h1要素内のstrong要素を取得する。
strong = h1.cssselect('strong')[0]
# text属性で要素のテキスト（より正確には開始タグ直後のテキスト）を取得する。
strong.text

'おいしい'

In [19]:
# tail属性で要素の直後のテキストを取得できる。
strong.tail

'今日のくだもの'

In [20]:
# h1要素は開始タグの直後に次の要素があり、テキストがないのでtext属性はNoneになる。
h1.text

In [21]:
# h1要素の直後の改行文字
h1.tail

'\n    '

In [22]:
# text_content()メソッドで要素内のすべてのテキストを結合した文字列を取得できる。
h1.text_content()

'おいしい今日のくだもの'

In [24]:
# scrape_by_lxml.py
# lxmlでスクレイピングする

import lxml.html

# HTMLファイルを読み込み、getroot()メソッドでHtmlElementオブジェクトを取得する。
tree = lxml.html.parse('dp.html')
html = tree.getroot()

# 引数のURLを基準として、すべてのa要素のhref属性を絶対URLに変換する。
html.make_links_absolute('https://gihyo.jp/')

# cssselect()メソッドで、セレクターに該当するa要素のリストを取得して、個々のa要素に対して処理を行う。
# セレクターの意味:
# id="listBook"である要素の子要素であるli要素の子要素であるitemprop="url"という属性を持つa要素
for a in html.cssselect('#listBook > li > a[itemprop="url"]'):
    # a要素のhref属性から書籍のURLを取得する。
    url = a.get('href')
    
    # 書籍のタイトルはitemprop="name"という属性を持つp要素から取得する。
    p = a.cssselect('p[itemprop="name"]')[0]
    # wbr要素などが含まれるのでtextではなくtext_content()を使う。
    title = p.text_content()
    
    # 書籍のURLとタイトルを出力する。
    print(url, title)

https://gihyo.jp/dp/ebook/2023/978-4-297-13486-0 今すぐ使えるかんたん 今すぐ使えるかんたんぜったいデキます！ パワーポイント超入門［Office 2021／Microsoft 365両対応］
https://gihyo.jp/dp/ebook/2023/978-4-297-13504-1 ゼロからはじめる ゼロからはじめるドコモ arrows N F-51C スマートガイド
https://gihyo.jp/dp/ebook/2023/978-4-297-13462-4 知りたい！サイエンス ゼータへの最初の一歩 ベルヌーイ数～「べき乗和」と素数で割った「余り」の驚くべき関係～
https://gihyo.jp/dp/ebook/2023/978-4-297-13442-6 今すぐ使えるかんたん 今すぐ使えるかんたんiPad完全ガイドブック 困った解決&便利技［iPadOS 16対応版］
https://gihyo.jp/dp/ebook/2023/978-4-297-13490-7 売れるランディングページ改善の法則
https://gihyo.jp/dp/ebook/2023/978-4-297-13241-5 ディープラーニングG検定（ジェネラリスト） 法律・倫理テキスト
https://gihyo.jp/dp/ebook/2023/978-4-297-13496-9 エンジニア選書 実践 Svelte入門
https://gihyo.jp/dp/ebook/2023/978-4-297-13510-2 現場が動きだす大学教育のマネジメントとは―茨城大学「教育の質保証」システム構築の物語
https://gihyo.jp/dp/ebook/2023/978-4-297-13508-9 Minecraftオフィシャルブック マインクラフト モブのたくらみ［石の剣のものがたりシリーズ②］
https://gihyo.jp/dp/ebook/2023/978-4-297-13408-2 図解即戦力 図解即戦力脱炭素のビジネス戦略と技術がこれ1冊でしっかりわかる教科書
https://gihyo.jp/dp/ebook/2023/978-4-297-13436-5 ノンプログラマーのための Visual Studio 

### 2-6. データをファイルに保存する

#### 2-6-1. CSV形式での保存

In [1]:
# save_csv_json.py
# シンプルにCSV形式で保存する。

# ヘッダーを書き出す
print('rank,city,population')
# join()メソッドの引数に渡すlistの要素はstrでなければならない
print('.'.join(['1', '上海', '24150000']))
print('.'.join(['2', 'カラチ', '23500000']))
print('.'.join(['3', '北京', '21516000']))
print('.'.join(['4', '天津', '14722100']))
print('.'.join(['5', 'イスタンブル', '14160467']))

rank,city,population
1.上海.24150000
2.カラチ.23500000
3.北京.21516000
4.天津.14722100
5.イスタンブル.14160467


In [4]:
# save_csv.py
# リストのリストをCSV形式で保存する

import csv

# ファイルを書き込み用に開く
# newline=''として改行コードの自動変換を抑制する
with open('top_cities.csv', 'w', newline='') as f:
    # csv.writeはファイルオブジェクトを引数に指定する
    write = csv.writer(f)
    # １行目のヘッダーを出力する
    write.writerow(['rank', 'city', 'population'])
    # writerows()で複数の行を一度に出力する
    # 引数はリストのリスト
    write.writerows([
        [1, '上海', 24150000],
        [2, 'カラチ', 23500000],
        [3, '北京', 21516000],
        [4, '天津', 14722100],
        [5, 'イスタンブル', 14160467],
    ])

In [6]:
# save_csv_dict.py
# 辞書のリストをCSV形式で保存する

import csv

with open('top_cities2.csv', 'w', newline='') as f:
    # 第１引数にファイルオブジェクト
    # 第２引数にフィールド名のリストを指定する
    writer = csv.DictWriter(f, ['rank', 'city', 'population'])
    # １行目のヘッダーを出力する
    writer.writeheader()
    # writerows()で複数の行を一度に出力する
    # 引数は辞書のリスト
    writer.writerows([
        {'rank': 1, 'city': '上海', 'population': 24150000},
        {'rank': 2, 'city': 'カラチ', 'population': 23500000},
        {'rank': 3, 'city': '北京', 'population': 21516000},
        {'rank': 4, 'city': '天津', 'population': 14722100},
        {'rank': 5, 'city': 'イスタンブル', 'population': 14160467},
    ])

#### 2-6-2. JSON形式での保存

In [10]:
# save_json.py
# JSON形式の文字列に変換する

import json

cities = [
    {'rank': 1, 'city': '上海', 'population': 24150000},
    {'rank': 2, 'city': 'カラチ', 'population': 23500000},
    {'rank': 3, 'city': '北京', 'population': 21516000},
    {'rank': 4, 'city': '天津', 'population': 14722100},
    {'rank': 5, 'city': 'イスタンブル', 'population': 14160467},
]

print(json.dumps(cities))

print('')

# ensure_ascii=False ASCII以外の文字を\uxxxxという形式でエスケープしないで出力する
# indent=2 適宜改行が挿入され２つの空白でインデントされる
print(json.dumps(cities, ensure_ascii=False, indent=2))

[{"rank": 1, "city": "\u4e0a\u6d77", "population": 24150000}, {"rank": 2, "city": "\u30ab\u30e9\u30c1", "population": 23500000}, {"rank": 3, "city": "\u5317\u4eac", "population": 21516000}, {"rank": 4, "city": "\u5929\u6d25", "population": 14722100}, {"rank": 5, "city": "\u30a4\u30b9\u30bf\u30f3\u30d6\u30eb", "population": 14160467}]

[
  {
    "rank": 1,
    "city": "上海",
    "population": 24150000
  },
  {
    "rank": 2,
    "city": "カラチ",
    "population": 23500000
  },
  {
    "rank": 3,
    "city": "北京",
    "population": 21516000
  },
  {
    "rank": 4,
    "city": "天津",
    "population": 14722100
  },
  {
    "rank": 5,
    "city": "イスタンブル",
    "population": 14160467
  }
]


### 2-7 Pythonによるスクレイピングの流れ

- main()関数
  - ３つの処理を順に呼び出す
- fetch(url: str) -> str
  - 引数urlで与えられたURLのWebページを取得する。
- scrape(html: str, base_url: str) -> List[dict]
  - 引数htmlで与えられたHTMLから正規表現で書籍の情報を抜き出す。
  - 引数base_urlは絶対URLに変換する際の基準となるURLを指定する。
- save(file_path: str, books: List[dict])
  - 引数booksで与えられた書籍のリストをCSV形式のファイルに保存する。

In [1]:
# python_scraper.py
# Pythonによるスクレイピング

%run python_scraper.py

In [2]:
%cat books.csv

url,title
https://gihyo.jp/dp/ebook/2023/978-4-297-13418-1,因果推論入門〜ミックステープ：基礎から現代的アプローチまで
https://gihyo.jp/dp/ebook/2023/978-4-297-13456-3,らくらく突破 らくらく突破第7版 貸金業務取扱主任者 ○×問題＋過去問題集
https://gihyo.jp/dp/ebook/2023/978-4-297-13482-2,デザインの学校 デザインの学校これからはじめる Illustrator & Photoshopの本［2023年最新版］
https://gihyo.jp/dp/ebook/2023/978-4-297-13486-0,今すぐ使えるかんたん 今すぐ使えるかんたんぜったいデキます！ パワーポイント超入門［Office 2021／Microsoft 365両対応］
https://gihyo.jp/dp/ebook/2023/978-4-297-13504-1,ゼロからはじめる ゼロからはじめるドコモ arrows N F-51C スマートガイド
https://gihyo.jp/dp/ebook/2023/978-4-297-13462-4,知りたい！サイエンス ゼータへの最初の一歩 ベルヌーイ数～「べき乗和」と素数で割った「余り」の驚くべき関係～
https://gihyo.jp/dp/ebook/2023/978-4-297-13442-6,今すぐ使えるかんたん 今すぐ使えるかんたんiPad完全ガイドブック 困った解決&便利技［iPadOS 16対応版］
https://gihyo.jp/dp/ebook/2023/978-4-297-13490-7,売れるランディングページ改善の法則
https://gihyo.jp/dp/ebook/2023/978-4-297-13241-5,ディープラーニングG検定（ジェネラリスト） 法律・倫理テキスト
https://gihyo.jp/dp/ebook/2023/978-4-297-13496-9,エンジニア選書 実践 Svelte入門
https://gihyo.jp/dp/ebook/2023/978-4-297-13510-2,現場が動きだす大学教育のマネジメントとは―茨城大

### 2-8. URLの基礎知識

#### 2-8-1. URLの構造

- `http://example.com/main/index?q=python#lead`
  - スキーム
    - `http`や`https`のようにプロトコルを表す
    - `http`
  - オーソリティ
    - `//`のあとに続き、通常ホスト名を表す。
    - ユーザー名やパスワード、ポート番号を含む場合もある。
    - `example.com`
  - パス
  - `/`で始まり、そのホストにおけるリソースのパスを表す。
    - `/main/index`
  - クエリ
    - `?`のあとに続き、パスとは異なる方法でリソースを指定するために使われる。
    - 存在しない場合もある。
    - `q=python`
  - フラグメント
    - `#`のあとに続き、リソース内の特定の部分などを表す。
    - 存在しない場合もある。
    - `lead`

#### 2-8-2. 絶対URLと相対URL

- 相対URL
  - `//`で始まるそうたいURL
  - `/`で始まるそうたいURL
  - それ以外の相対URL形式の相対URL

In [10]:
# 相対URLから絶対URLへの返還

from urllib.parse import urljoin

base_url = 'http://example.com/books/top.html'

In [11]:
# `//`で始まる相対URL
urljoin(base_url, '//cdn.example.com/logo.png')

'http://cdn.example.com/logo.png'

In [12]:
# `/`で始まる相対URL
urljoin(base_url, '/articles/')

'http://example.com/articles/'

In [13]:
# 相対パス形式の相対URL
urljoin(base_url, './')

'http://example.com/books/'